https://www.youtube.com/watch?v=enObIMzyaE4

In [ ]:
!pip install transformers torch datasets

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
# Login to huggingface to upload the fine tuned model using a write token
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import Dataset, Audio # Import the Audio class

Local files example

In [43]:
Necaxa_TTS = Dataset.from_dict({"audio": ["/content/Necaxa_1/data/Inundacion-1.wav", "/content/Necaxa_1/data/Inundacion_nec100_1.wav",
                                        "/content/Necaxa_1/data/Inundacion-2.wav", "/content/Necaxa_1/data/Inundacion_nec100_2.wav",
                                        "/content/Necaxa_1/data/Inundacion-3.wav", "/content/Necaxa_1/data/Inundacion_nec100_3.wav",
                                        "/content/Necaxa_1/data/Inundacion-4.wav", "/content/Necaxa_1/data/Inundacion_nec100_4.wav",
                                        "/content/Necaxa_1/data/Inundacion-5.wav", "/content/Necaxa_1/data/Inundacion_nec100_5.wav",
                                        "/content/Necaxa_1/data/Inundacion-6.wav", "/content/Necaxa_1/data/Inundacion_nec100_6.wav",
                                        "/content/Necaxa_1/data/Inundacion-7.wav", "/content/Necaxa_1/data/Inundacion_nec100_7.wav",
                                        "/content/Necaxa_1/data/Inundacion-8.wav", "/content/Necaxa_1/data/Inundacion_nec100_8.wav",
                                        "/content/Necaxa_1/data/Inundacion-9.wav", "/content/Necaxa_1/data/Inundacion_nec100_9.wav",
                                        "/content/Necaxa_1/data/Inundacion-10.wav", "/content/Necaxa_1/data/Inundacion_nec100_10.wav",
                                        "/content/Necaxa_1/data/Inundacion-11.wav", "/content/Necaxa_1/data/Inundacion_nec100_11.wav",
                                        "/content/Necaxa_1/data/Inundacion-12.wav", "/content/Necaxa_1/data/Inundacion_nec100_12.wav",
                                        "/content/Necaxa_1/data/Inundacion_1-12_16.wav", "/content/Necaxa_1/data/Inundacion_nec100_1-12.wav", "/content/Necaxa_1/data/Inundacion_nec100_1-12_base.wav",
                                        "/content/Necaxa_1/data/historia_de_maiz_01.wav", "/content/Necaxa_1/data/historia_del_maiz.wav", "/content/Necaxa_1/data/historia_del_maiz_base.wav",
                                        "/content/Necaxa_1/data/vibora_voladora_orig.wav", "/content/Necaxa_1/data/vibora_voladora.wav", "/content/Necaxa_1/data/vibora_voladora_base.wav"]
                              }).cast_column("audio", Audio())

In [21]:
inundacion = ("hee chaa chuúkáa' tzamáá" + "   " +
                "cuento xla tzamáá" + "   " +
                "xni munkaa'lakaníí' tii chuu kawa ix tuu xlhoomááka'" + "   " +
                "aa xla akxní mat chaatín chi'xkú' kwan nakaa'takúxtu'" + "   " +
                "tuu chii aamáá mat chii a'j skujááj" + "   " +
                "mat ixá'n skúja" + "   " +
                "tuu chaa chii wa' ne'hnina'" + "   " +
                "hee mat xla'hkaanamaalhtzá tzamáá chi'xkú'" + "   " +
                "hee mat xchaampalá kaa'tzi'sájna' yaapalahoolhtzá kí'wi'" + "   " +
                "xchaampalá kaa'tzi'sájna' yaapalahoolhtzá kí'wi" + "   " +
                "hee mat wan tzamáá chi'xkú'" + "   " +
                "tii tzaj tzeeyáá uutzá chuuntzá kileehamaanán mat wan")

In [22]:
historia = ("pus juu, wajtzananú jaa tii ixkatzíi palh ixwíilh tzamáá kúxi'" + "   " +
                 "jaa tii ixkatzíí palh ixwakán kúxi'" + "   " +
                 "pero milh chaatín chi'xkú' hee tzamáá manó'hlhli' tzamáá chaaán" + "   " +
                 "xala'há'lha' chaaán" + "   " +
                 "taleemaa'nálh tzamáá kúxi'  taleemaa'nálh nakixchikán")

In [23]:
vibora = ("ma'háá's tzamáá ixmín tzamáá la'hatín lúúwa'" + "   " +
                 "mat ixilí ixmín" + "   " +
                 "ixwiliinikán chaatín tzumaját" + "   " +
                 "uutzá ixléén" + "   " +
                 "uutzá ixléén akxní ixmín"  + "   " +
                 "pus aaliistáán tzamáá" + "   " +
                 "mat milh chaatín hawácha'" + "   " +
                 "mat xla'hlhaa'wáán" + "   " +
                 "mat tzaj tii kaaa'hxte'hma'háánka' kaa'ki'wí'n tzamáá hawácha'" + "   " +
                 "chuuwá aaliistáán ixtaatá' tii leelh" + "   " +
                 "kapu'tzayááuj malhát, mat waní" + "   " +
                 "leelh xtaatá'"
                 );

In [25]:
transcription = ["hee chaa chuúkáa' tzamáá" , "hee chaa chuúkáa' tzamáá" ,
                "cuento xla tzamáá" , "cuento xla tzamáá" ,
                "xni munkaa'lakaníí' tii chuu kawa ix tuu xlhoomááka'" , "xni munkaa'lakaníí' tii chuu kawa ix tuu xlhoomááka'" ,
                "aa xla akxní mat chaatín chi'xkú' kwan nakaa'takúxtu'" , "aa xla akxní mat chaatín chi'xkú' kwan nakaa'takúxtu'" ,
                "tuu chii aamáá mat chii a'j skujááj" , "tuu chii aamáá mat chii a'j skujááj" ,
                "mat ixá'n skúja" , "mat ixá'n skúja" ,
                "tuu chaa chii wa' ne'hnina'" , "tuu chaa chii wa' ne'hnina'" ,
                "hee mat xla'hkaanamaalhtzá tzamáá chi'xkú'" , "hee mat xla'hkaanamaalhtzá tzamáá chi'xkú'" ,
                "hee mat xchaampalá kaa'tzi'sájna' yaapalahoolhtzá kí'wi'" , "hee mat xchaampalá kaa'tzi'sájna' yaapalahoolhtzá kí'wi'" ,
                "xchaampalá kaa'tzi'sájna' yaapalahoolhtzá kí'wi" , "xchaampalá kaa'tzi'sájna' yaapalahoolhtzá kí'wi" ,
                "hee mat wan tzamáá chi'xkú'" , "hee mat wan tzamáá chi'xkú'" ,
                "tii tzaj tzeeyáá uutzá chuuntzá kileehamaanán mat wan", "tii tzaj tzeeyáá uutzá chuuntzá kileehamaanán mat wan",
                 inundacion, inundacion, inundacion,
                 historia, historia, historia,
                 vibora, vibora, vibora]

In [ ]:
transcription

In [ ]:
label = ["Inundacion-1","Inundacion-1_TTS",
         "Inundacion-2","Inundacion-2_TTS",
         "Inundacion-3","Inundacion-3_TTS",
         "Inundacion-4","Inundacion-4_TTS",
         "Inundacion-5","Inundacion-5_TTS",
         "Inundacion-6","Inundacion-6_TTS",
         "Inundacion-7","Inundacion-7_TTS",
         "Inundacion-8","Inundacion-8_TTS",
         "Inundacion-9","Inundacion-9_TTS",
         "Inundacion-10","Inundacion-10_TTS",
         "Inundacion-11","Inundacion-11_TTS",
         "Inundacion-12","Inundacion-12_TTS",
         "Inundacion-1-12","Inundacion-1-12_TTS","Inundacion-1-12_Nahuatl",
         "historia del maiz","historia del maiz_TTS","historia del maiz_Nahuatl",
         "vibora voladora","vibora voladora_TTS","vibora voladora_Nahuatl"]

In [ ]:
line_id = list(range(1, 101))
line_id

In [45]:
Necaxa_TTS = Necaxa_TTS.add_column("Source", label)


In [46]:
Necaxa_TTS = Necaxa_TTS.add_column("Transcription", transcription)

In [47]:
Necaxa_TTS

Dataset({
    features: ['audio', 'Source', 'Transcription'],
    num_rows: 33
})

In [ ]:
Necaxa_100[10]

{'audio': {'path': '/content/Necaxa_1/data/Inundacion-21.wav',
  'array': array([ 0.00137329,  0.00512695,  0.00588989, ..., -0.00189209,
         -0.00119019, -0.00094604]),
  'sampling_rate': 16000},
 'text': "mat lamá:lh ásta la'há: chi: lasti:má:lh mat ka:cha:nikma:pí: kí'wi' mat laks",
 'speaker_id': 1,
 'line_id': 11}

In [ ]:
Necaxa_TTS.push_to_hub("plesniar/Necaxa_TTS")